# Illustration 4.1

In this notebook, we provide an explanation with a toy example of the *Constraint 2* forbidding potential redundancies in a cell class (see Section 4.2.3 of the manuscript).

In the figure bellow, we recapitulate presented patterns of redundancies in the manuscript, as known V-patterns, parallel-patterns and cross-patterns.

![Redundant patterns](./figs/redundant_patterns.png)

**Tips**: to run an ASP program, click on the cell and then on `Run` (or press `Ctrl+Enter`). The result will be printed below the cell.

## V-patterns

We present here the ASP program of the toy example on the two constraints of lines *v2_10* and *v2_11*.

In [15]:
%%clingo -n 0 

% Facts of the toy example
match(a1,b1). match(a1,b2). 
match(a2,b3). match(a3,b3).

% Constraints of the program
:- match(I,J1), match(I,J2), J1!=J2.
:- match(I1,J), match(I2,J), I1!=I2.  


% Print of the results
#show match/2.

clingo version 5.4.1
Reading from stdin
Solving...
UNSATISFIABLE

Models       : 0
Calls        : 1
Time         : 0.000s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


In the first program, all matches are presents. However, given the constraints, it is impossible to have in the same time two matches involving the same cell.
Thus, the solver cannot find a solution, the problem is `UNSATISFIABLE`.

To be more similar with real problem, we modify the previous program by using choice rules, in order to consider or not a match.

In [16]:
%%clingo -n 0 

% Facts of the toy example
% Here, we use a choice rule enables the inference 0 or 1 time of each match
0{match(a1,b1)}1. 0{match(a1,b2)}1. 
0{match(a2,b3)}1. 0{match(a3,b3)}1. 

% Constraints of the program
:- match(I,J1), match(I,J2), J1!=J2.
:- match(I1,J), match(I2,J), I1!=I2.  

% Print of the results
#show match/2.

clingo version 5.4.1
Reading from stdin
Solving...
Answer: 1

Answer: 2
match(a3,b3)
Answer: 3
match(a1,b1)
Answer: 4
match(a3,b3) match(a1,b1)
Answer: 5
match(a1,b2)
Answer: 6
match(a3,b3) match(a1,b2)
Answer: 7
match(a2,b3)
Answer: 8
match(a2,b3) match(a1,b1)
Answer: 9
match(a2,b3) match(a1,b2)
SATISFIABLE

Models       : 9
Calls        : 1
Time         : 0.000s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


With this program, we obtain 9 solutions composed of 0, 1 or 2 matches.

---

## Parallel-patterns and Cross-patterns

We present here the ASP program of the toy example on parallel-patterns and cross-patterns.

First, we present the obtained patterns after the use of constraints $v2\_10$ and $v2\_11$ (see last cell of the notebook).

In [17]:
%%clingo 0 --opt-mode=enum,-2

% Facts of the toy example
pot_match(a1,b1). pot_match(a1,b2).
pot_match(a2,b1). pot_match(a2,b2).
0{match(I,J)}1 :- pot_match(I,J).

% Constraints of the program
:- match(I,J1), match(I,J2), J1!=J2.
:- match(I1,J), match(I2,J), I1!=I2.  

% Optimization
#maximize{1, I : match(I,_)}.

% Print of the results
#show match/2.

clingo version 5.4.1
Reading from stdin
Solving...
Answer: 1
match(a1,b2) match(a2,b1)
Optimization: -2
Answer: 2
match(a1,b1) match(a2,b2)
Optimization: -2
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
Calls        : 1
Time         : 0.000s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


Given the constraints on the `match/2` predicates, two possibles solutions comprising two matches are found. 
The first `match(a1,b2) match(a2,b1)` correspond to the cross-pattern illustrated in the above figure (B).
The second `match(a1,b1) match(a2,b2)` correspond to the parallel-pattern illustrated in the above figure (B).
However, these matches involved redundant cells showing the need of additional constraints.

To overcome this, we introduce two constraints (lines $v2\_12$ and $v2\_13$ in the program -- see last cell of the notebook).

In [23]:
%%clingo 0 

% Facts of the toy example
pot_match(a1,b1). pot_match(a1,b2).
pot_match(a2,b1). pot_match(a2,b2).
0{match(I,J)}1 :- pot_match(I,J).

% Constraints of the program
:- match(I,J1), match(I,J2), J1!=J2.
:- match(I1,J), match(I2,J), I1!=I2.  

:-pot_match(I1,J), pot_match(I2,J), match(I2,J), I1<I2.
:-pot_match(I,J1), pot_match(I,J2), match(I,J2), J1<J2.

% Optimization
#maximize{1, I : match(I,_)}.

% Print of the results
#show match/2.

clingo version 5.4.1
Reading from stdin
Solving...
Answer: 1

Optimization: 0
Answer: 2
match(a1,b1)
Optimization: -1
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
Optimization : -1
Calls        : 1
Time         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.001s


We observe that one solution satisfy the problem, comprising `match(a1,b1)`.


Let us observe the results if we apply the constraints $v2\_12$ and $v2\_13$, without considering the constraints $v2\_10$ and $v2\_11$ to the configuration A.

In [25]:
%%clingo 0 

% Facts of the toy example
pot_match(a1,b1). pot_match(a1,b2). 
pot_match(a2,b3). pot_match(a3,b3).
0{match(I,J)}1 :- pot_match(I,J).

% Constraints of the program
% :- match(I,J1), match(I,J2), J1!=J2.
% :- match(I1,J), match(I2,J), I1!=I2.  

:-pot_match(I1,J), pot_match(I2,J), match(I2,J), I1<I2.
:-pot_match(I,J1), pot_match(I,J2), match(I,J2), J1<J2.

% Optimization
#maximize{1, I : match(I,_)}.

% Print of the results
#show match/2.

clingo version 5.4.1
Reading from stdin
Solving...
Answer: 1

Optimization: 0
Answer: 2
match(a2,b3)
Optimization: -1
Answer: 3
match(a1,b1) match(a2,b3)
Optimization: -2
OPTIMUM FOUND

Models       : 3
  Optimum    : yes
Optimization : -2
Calls        : 1
Time         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.001s


Only one solution `match(a1,b1) match(a2,b3)` is allowed by the constraints.

---

In the following cell, we provide the $v2$ program for identifying pseudo-perturbations (see Program 4.1 in the manuscript).

```
{selinput(G) : pert(C,G,S,CL), not intermediate(G)} = l.
{selinter(G): intermediate(G)} = k-l.
selpert(E,V,S,C) :- selinput(V), pert(E,V,S,C).
selpert(E,V,S,C) :- selinter(V), pert(E,V,S,C).
equal(I,J,G) :- selpert(I,G,S1,C1), selpert(J,G,S2,C2), I!=J, S1=S2.
pot_match(I,J) :- k = {equal(I,J,_)},  selpert(I,_,_,C1), selpert(J,_,_,C2), C1<C2, I!=J.
0{match(I,J)}1 :- pot_match(I,J).
nbInputOnes(C,N) :- N = {pert(C,G,1,_) : selinput(G), input(G)}, match(C,_).
:- match(C,_), nbInputOnes(C,N), N < 1.  
:- match(I,J1), match(I,J2), J1!=J2.
:- match(I1,J), match(I2,J), I1!=I2.  
:- pot_match(I1,J), pot_match(I2,J), match(I2,J), I1<I2.
:- pot_match(I,J1), pot_match(I,J2), match(I,J2), J1<J2.
1{selinput(G)  : parent(G,I), input(G)} :- selinter(I).
#maximize{1, I : match(I,_)}.
```